## OpenAI (ChatGPT)

Beispiel für die Erstellung einer Python Applikation

Zuerst muss der Open AI API Key erstellt und ein Guthaben bereitgestellt werden:
* [API Key erstellen](https://platform.openai.com/account/api-keys)
* [Guthaben bereitstellen](https://platform.openai.com/settings/organization/billing/overview)


In [ ]:
import os
from openai import OpenAI
from IPython.display import Markdown, display
from datetime import datetime

os.environ['OPENAI_API_KEY']=''

Um den Prozess zu automatisieren, erstellen wir eine Funktion, die eine neue Benutzeranfrage entgegennimmt, sie an die KI sendet und die Antwort zurückgibt.

Allgemeine Anweisungen, wie Output im Markdown-Format etc., geben wir in `messages` mit.

In [ ]:
# OpenAI-Client initialisieren
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Konversationsverlauf starten
messages = [
    {"role": "system", "content": '''
    Bitte formatiere die Antwort im Markdown-Format
    Verwende Schweizer Schriftsprache bei den Antworten
    '''}
]

def chat_with_openai(user_input, messages, client):
    messages.append({"role": "user", "content": user_input})
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
    )
    assistant_message = response.choices[0].message.content
    messages.append({"role": "assistant", "content": assistant_message})
    return assistant_message


Ein Beispielaufruf sieht dann wie folgt aus:

In [ ]:
user_input = "Wie unterscheidet sich Prozessoptimierung von Prozessmanagement?"

response = chat_with_openai(user_input, messages, client)

# Antwort anzeigen
display(Markdown(response))

- - -

## Prompts um eine Webabwendung zu erstellen

Als erstes brauchen wir eine Funktion `parse_response_to_files` um den Output von OpenAI in Dateien umzuwandeln


In [ ]:
import re
import os
from pathlib import Path

def parse_response_to_files(response_text: str, output_dir="webshop"):
    """
    Parst OpenAI Markdown-Antworten und extrahiert Dateinamen + zugehörige Codeblöcke.
    Erkennt sowohl '### ... (filename.ext)' als auch '#### filename.ext'.
    """
    os.makedirs(output_dir, exist_ok=True)

    # Nur zulässige Dateinamen erkennen
    valid_extensions = (".py", ".sh", ".json", ".yaml", ".yml", ".txt", ".env", ".cfg")
    filename_pattern_1 = re.compile(r"^#+ .*?\(([^)]+)\)", re.MULTILINE)  # z.B. (Docker.customer)
    filename_pattern_2 = re.compile(r"^####\s+([\w\-\.]+)$", re.MULTILINE)  # nur wenn direkt Dateiname

    filenames_1 = filename_pattern_1.findall(response_text)
    filenames_2 = [
        fn for fn in filename_pattern_2.findall(response_text)
        if fn == "Dockerfile" or fn.endswith(valid_extensions)
    ]
    filenames = filenames_1 + filenames_2

    # Codeblöcke extrahieren
    codeblocks = re.findall(r"```(?:\w+)?\n(.*?)```", response_text, re.DOTALL)

    if len(filenames) != len(codeblocks):
        print("⚠️ Anzahl der Dateinamen stimmt nicht mit Anzahl der Codeblöcke überein.")
        print(f"Gefundene Dateinamen: {len(filenames)}, Codeblöcke: {len(codeblocks)}")

    written_files = []

    for name, code in zip(filenames, codeblocks):
        filepath = Path(output_dir) / name
        with open(filepath, "w", encoding="utf-8") as f:
            f.write(code.strip() + "\n")
        written_files.append(str(filepath))

    print(f"✅ {len(written_files)} Dateien gespeichert:")
    for f in written_files:
        print(f"  - {f}")

    return written_files


Die OpenAI-API ist zustandslos, was bedeutet, dass sie sich nicht an vorherige Interaktionen erinnert. 

Um dennoch einen Gesprächsverlauf zu simulieren, verwenden wir eine Liste (`messages`), die sowohl Benutzer- als auch KI-Nachrichten enthält.

Weitere Prompts siehe [hier](https://gitlab.com/ch-mc-b/autoshop-ms/edu/chatgpt)

In [ ]:
# Konversationsverlauf starten
messages = [
    {"role": "system", "content": '''
    Bitte formatiere die Antwort im Markdown-Format
    Verwende Schweizer Schriftsprache bei den Antworten
    Bitte gib mir die Codeblöcke jeweils im folgenden Format zurück:
    ### Beschreibung (filename.py)
    ```python
    code
    ```
    '''}
]

user_input = '''
Basierend auf den Microservices Beispiel von Eberhard Wolff schreibe mit 3 Python Scripte welche jeweils eine Webseite mit
- Kundendaten (Customer)
- Produktdaten (Catalog)
- Bestellungen (Order)
enthalten.
Die Scripte sollen die Daten via REST-API /<service>/api und im HTML-Format /<service/ zur Verfügung stellen.

Die Produkte sollen dem eines Auto Hauses mit Autos und Auto Zubehör Artikeln entsprechen.

Als TCP/IP Port soll generell 8080 und die Services sollen gegen 0.0.0.0 geöffnet sein.
'''

response = chat_with_openai(user_input, messages, client)

# Antwort anzeigen
display(Markdown(response))
parse_response_to_files(response)

Mit dem Grundgerüst der Applikation können wir weiterfahren und das ganze als Container Images aufbereiten.

In [ ]:
user_input = '''
Erstelle für die Services ein Dockerfile und eine Docker-Compose Datei.
Bezeichne die Dockerfile mittels Docker.<service>
'''

response = chat_with_openai(user_input, messages, client)

# Antwort anzeigen
display(Markdown(response))
parse_response_to_files(response)


- - - 
## Applikation bauen

Zum Schluss bauen wir die Applikation zusammen und 


In [ ]:
%%bash
cd webshop
docker-compose build


und starten die Applikation

In [ ]:
%%bash
echo "Customer: http://$(cat ~/work/server-ip):8080/customer"
echo "Catalog : http://$(cat ~/work/server-ip):8081/catalog"
echo "Order   : http://$(cat ~/work/server-ip):8082/order"
cd webshop
docker-compose up